In [98]:
from datetime import date, timedelta
from data_provider import DataModel
import pandas as pd
import numpy as np
import abc
from scipy import stats

class signal(abc.ABC):
    @abc.abstractmethod
    def get_signals(symbol_data):
        pass
    
def profit_in_interval(symbol_data, signal_function):
    signals = signal_function(symbol_data)
    buy_profits = []
    buy_profits_length = []
    sell_profits = []
    sell_profits_length = []
    for date, length, sell_or_buy in signals:
        first_price = symbol_data.loc[date]["close"]
        buy = []
        sell = []
        df_close = symbol_data[date+timedelta(days=1):date+timedelta(days=length)]["close"]
        if len(df_close) > 0:
            if sell_or_buy:
                buy_profits.append((df_close.max() - first_price) / first_price)
                buy_profits_length.append((df_close.idxmax() - date).days)
            else:
                sell_profits.append((df_close.min() - first_price) / first_price)
                sell_profits_length.append((df_close.idxmin() - date).days)
        
    if len(buy_profits) == 0:
        buy_profits = [0]
    if len(buy_profits_length) == 0:
        buy_profits_length = [0]
    if len(sell_profits) == 0:
        sell_profits = [0]
    if len(sell_profits_length) == 0:
        sell_profits_length = [0]
    
    buy_profits = np.array(buy_profits)
    buy_profits_length = np.array(buy_profits_length)
    sell_profits = np.array(sell_profits)
    sell_profits_length = np.array(sell_profits_length)
    
    ans = [(buy_profits.mean() if len(buy_profits) > 0 else 0,
            buy_profits_length.mean() if len(buy_profits_length) > 0 else 0),
           (sell_profits.mean() if len(sell_profits) > 0 else 0,
            sell_profits_length.mean() if len(sell_profits_length) > 0 else 0)]
    ans = [(stats.describe(buy_profits), stats.describe(buy_profits_length)),
           (stats.describe(sell_profits), stats.describe(sell_profits_length))]
    return ans



def minus_signal(symbol_data, day_length, future_length):
    signals = []
    count = 0
    for date in symbol_data.index:
        if symbol_data.loc[date, "ending-percent"] < -3:
            count = count + 1
        else:
            if symbol_data.loc[date, "ending-percent"] > 0:
                if count >= day_length:
#                     print(date)
                    signals.append((date, future_length, True))
            count = 0
    return signals


def crossover(series1, series2):
    return pd.Series(np.where(series1 > series2, 1.0, 0.0)).diff()
def SMA_signal(symbol_data, short_length, long_length, future_length):
    def calcSma(data, smaPeriod):
        j = next(i for i, x in enumerate(data) if x is not None)
        our_range = range(len(data))[j + smaPeriod - 1:]
        empty_list = [None] * (j + smaPeriod - 1)
        sub_result = [np.mean(data[i - smaPeriod + 1: i + 1]) for i in our_range]
        return np.array(empty_list + sub_result)
    
    def ema(data, days):
        return data.ewm(span=days).mean()
    
    def sma(data, days):
        return data.rolling(window=days).mean()
    
    signals = []
    close = pd.Series(symbol_data["close"])
    ssma = ema(close, short_length)
    lsma = ema(close, long_length)
    
    cross = crossover(ssma, lsma)
    cross.index = symbol_data.index
    buy = list(cross[cross > 0.0].index)
    sell = list(cross[cross < 0.0].index)
    for b in buy:
        signals.append((b, future_length, True))
    for s in sell:
        signals.append((s, future_length, False))
    return signals


def stochastic_signal(symbol_data, d, k, future_length):
    x=TA.STOCHD(my_df)
    y=TA.STOCH(my_df)
    self.st_d = self.I(TA.STOCHD,my_df)
    self.st_k = self.I(TA.STOCH,my_df)
     if crossover(self.st_d, self.st_k) and self.st_d>=80:
            self.buy()
    elif crossover(self.st_k, self.st_d) and self.st_d<=20:
            self.sell()

In [101]:
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

%matplotlib inline

pd.options.mode.chained_assignment = 'raise'

def sma1545(x):
    return SMA_signal(x, 15, 50, 90)

buys_lens = [0]
buys = []
sells_lens = [0]
sells = []


for symbol in tqdm(dm.TA_SYMBOLS):
    df = dm.get(symbol)
    buy_lens = []
    buy = []
    sell_lens = []
    sell = []
    for s in range(10, 20):
        for l in range(40, 50):
            def smasig(x):
                return SMA_signal(x, s, l, 45)
            res = profit_in_interval(df, smasig)
            buy.append([s, l, res[0][0].mean])
            buy_lens.append([s, l, res[0][1].mean])
            sell.append([s, l, res[1][0].mean])
            sell_lens.append([s, l, res[1][1].mean])
    buy = np.array(buy)
    sell = np.array(sell)
    buys.append(buy[np.argmax(buy, axis = 0)[2]])
    sells.append(sell[np.argmax(sell, axis = 0)[2]])

print(sells)
print(buys)
# print(f'overall: {stats.describe(buys)}')
# print(f"overall:  {stats.describe(buys_lens)}")
# print(f'overall:  {stats.describe(sells)}')
# print(f'overall:  {stats.describe(sells_lens)}')

100%|██████████| 16/16 [01:49<00:00,  6.86s/it]

[array([19.        , 44.        , -0.06585082]), array([10.        , 42.        , -0.09947932]), array([15.        , 41.        , -0.08421309]), array([19.        , 49.        , -0.07745079]), array([ 1.70000000e+01,  4.00000000e+01, -3.90943459e-02]), array([17.        , 44.        , -0.08408884]), array([ 1.90000000e+01,  4.90000000e+01, -4.76009176e-02]), array([10.        , 41.        , -0.05689034]), array([19.        , 41.        , -0.08791469]), array([17.        , 49.        , -0.06743485]), array([ 1.40000000e+01,  4.90000000e+01, -4.74657324e-02]), array([10.        , 41.        , -0.07705426]), array([19.        , 44.        , -0.06585082]), array([10.        , 44.        , -0.06946047]), array([ 1.80000000e+01,  4.90000000e+01, -4.49615993e-02]), array([ 1.3000000e+01,  4.8000000e+01, -1.5707422e-02])]
[array([12.       , 48.       ,  0.2192493]), array([19.        , 47.        ,  0.20575757]), array([16.        , 49.        ,  0.26503322]), array([15.        , 41.        ,

In [102]:
tmpb = []
for a in buys:
    tmpb.append((a[0], a[1], a[2]))
for a in tmpb:
    print(a)

(12.0, 48.0, 0.21924929758143094)
(19.0, 47.0, 0.20575757162961253)
(16.0, 49.0, 0.2650332191326107)
(15.0, 41.0, 0.2554486885530741)
(18.0, 42.0, 0.1539212136634982)
(18.0, 48.0, 0.23333513315157614)
(11.0, 48.0, 0.20052698821288795)
(10.0, 41.0, 0.24521682105846215)
(15.0, 41.0, 0.2033777770339337)
(13.0, 47.0, 0.23109194048173334)
(10.0, 40.0, 0.13988527861179892)
(10.0, 40.0, 0.19943237189887092)
(12.0, 48.0, 0.21924929758143094)
(18.0, 41.0, 0.14097871416094931)
(19.0, 41.0, 0.28696370211946237)
(10.0, 44.0, 0.14536532135369176)


In [38]:


def minus_signal_2(x):
    return minus_signal(x, 3, 15)

for symbol in dm.TA_SYMBOLS:
    df = dm.get(symbol)
    for stat in profit_in_interval(df, minus_signal_2):
        print(f'{symbol}\n{stat[0]}\n{stat[1]}')
    print("---------------")
    
# print(df.loc[:, "ending-percent"])

خپارس
DescribeResult(nobs=6, minmax=(0.05006954102920723, 0.20724465558194774), mean=0.09051817490452745, variance=0.0035082762172046747, skewness=1.5270755490805636, kurtosis=0.7226271385065548)
DescribeResult(nobs=6, minmax=(5, 9), mean=7.166666666666667, variance=1.7666666666666664, skewness=-0.32137188546485806, kurtosis=-0.3994304022783908)
خپارس
DescribeResult(nobs=1, minmax=(0, 0), mean=0.0, variance=nan, skewness=0.0, kurtosis=-3.0)
DescribeResult(nobs=1, minmax=(0, 0), mean=0.0, variance=nan, skewness=0.0, kurtosis=-3.0)
---------------
خكاوه
DescribeResult(nobs=11, minmax=(-0.028272251308900525, 0.25806451612903225), mean=0.07514014897676233, variance=0.008084968486950134, skewness=0.706082135801913, kurtosis=-0.5124013996691241)
DescribeResult(nobs=11, minmax=(1, 15), mean=8.909090909090908, variance=21.490909090909092, skewness=-0.28674089805551795, kurtosis=-1.2686876755391792)
خكاوه
DescribeResult(nobs=1, minmax=(0, 0), mean=0.0, variance=nan, skewness=0.0, kurtosis=-3.0)

In [5]:
from data_provider import DataModel

dm = DataModel("../../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [6]:
df = dm.get(dm.TA_SYMBOLS[0])
date = df.index[2]
(df.loc[date:date+timedelta(days=10), "close"].idxmax() - date).days

/home/nimac/.local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


3